In [ ]:
!pip install boto3 --upgrade

In [1]:
import boto3
import json
sagemaker_client = boto3.client('runtime.sagemaker')
bedrock_client = boto3.client('bedrock-runtime')


In [72]:
def preprocess_prompt_claude(event, language, writing_style):
    system_styles = {
        "Twitter": f"You are a live ticker and soccer expert and able to create twitter tweets with emoji",
        "Commentator": f"You are a live ticker and soccer expert",
    }
    if writing_style not in system_styles:
        raise ValueError("Unsupported writing style")
    
    languages = {
        "German": "in german language",
        "English": "",
        "French": "in french language",
        "Spanish": "in spanish language",
        "Arabic": "in arabic language",
        "Traditional Chinese": "in traditional chinese language",
        "Turkish": "in turkish language",
        "Dutch": "in dutch language"

    }
    if language not in languages:
        raise ValueError("Unsupported language")

    
    sys_content = system_styles[writing_style] + " " + languages[language]

    content = f"players shoot at the goal. the aim is to shoot as hard as possible and at a high shooting speed. The system generates the following attributes for each shot in a json. shot_speed is measured in km/h. shot_distance in meters. shot_goal is true if the goal was hit, otherwise false. leaderboard_ranking indicates where this shot is now in the ranking. shots_done is the number of shots this person has already made. generate engaging commentary in one sentence."

    prompt = f'''Human: {sys_content}
    <event>
    {event}
    </event>
    {content}
    Do not introduce your output. write in the present. Write this in an exciting style just as a soccer commentator would.
    Assistant:'''
    return {"prompt": prompt,
        "max_tokens_to_sample":100,
        "temperature":0.3,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences":[]}

In [73]:
def call_llm_bedrock_claude_endpoint(payload, model_type):
    model_id_mapping = {
        'claude-v1': 'anthropic.claude-v1',
        'claude-v2': 'anthropic.claude-v2',
        'claude-instant': 'anthropic.claude-instant-v1'
    }
    
    model_id = model_id_mapping.get(model_type)
    if not model_id:
        raise ValueError(f"Invalid model_type: {model_type}")

    accept = 'application/json'
    content_type = 'application/json'
    body = json.dumps(payload)
    response = bedrock_client.invoke_model(body=body, modelId=model_id, accept=accept, contentType=content_type)
    result = json.loads(response.get('body').read())
    return result.get('completion')


In [74]:
example_data = [
  {
    "player_name": "John",
    "shot_speed": 120,
    "shot_distance": 25,
    "shot_goal": True,
    "leaderboard_ranking": 1,
    "shots_done": 15
  },
  {
    "player_name": "Sarah",
    "shot_speed": 110,
    "shot_distance": 20,
    "shot_goal": False,
    "leaderboard_ranking": 2,
    "shots_done": 18
  },
  {
    "player_name": "Mike",
    "shot_speed": 125,
    "shot_distance": 28,
    "shot_goal": True,
    "leaderboard_ranking": 3,
    "shots_done": 20
  },
  {
    "player_name": "Emily",
    "shot_speed": 115,
    "shot_distance": 22,
    "shot_goal": True,
    "leaderboard_ranking": 4,
    "shots_done": 14
  },
  {
    "player_name": "Alex",
    "shot_speed": 130,
    "shot_distance": 30,
    "shot_goal": True,
    "leaderboard_ranking": 5,
    "shots_done": 17
  },
  {
    "player_name": "Sophia",
    "shot_speed": 105,
    "shot_distance": 18,
    "shot_goal": False,
    "leaderboard_ranking": 6,
    "shots_done": 22
  },
  {
    "player_name": "Daniel",
    "shot_speed": 128,
    "shot_distance": 29,
    "shot_goal": True,
    "leaderboard_ranking": 7,
    "shots_done": 19
  },
  {
    "player_name": "Olivia",
    "shot_speed": 112,
    "shot_distance": 21,
    "shot_goal": True,
    "leaderboard_ranking": 8,
    "shots_done": 16
  },
  {
    "player_name": "Liam",
    "shot_speed": 118,
    "shot_distance": 24,
    "shot_goal": True,
    "leaderboard_ranking": 9,
    "shots_done": 21
  },
  {
    "player_name": "Ava",
    "shot_speed": 123,
    "shot_distance": 27,
    "shot_goal": True,
    "leaderboard_ranking": 10,
    "shots_done": 18
  }
]


In [75]:
payload = preprocess_prompt_claude(example_data[1], "Dutch", "Commentator")

In [76]:
example_data[1]

{'player_name': 'Sarah',
 'shot_speed': 110,
 'shot_distance': 20,
 'shot_goal': False,
 'leaderboard_ranking': 2,
 'shots_done': 18}

In [78]:
%%time
call_llm_bedrock_claude_endpoint(payload,"claude-instant")

CPU times: user 3.42 ms, sys: 0 ns, total: 3.42 ms
Wall time: 1.01 s


' Sarah schiet hard maar mist net, met een snelheid van 110 kilometer per uur vanaf 20 meter! Ze blijft op de tweede plaats van de ranglijst staan met 18 geschoten pogingen.'

In [68]:
%%time
call_llm_bedrock_claude_endpoint(payload,"claude-v2")

CPU times: user 4.91 ms, sys: 0 ns, total: 4.91 ms
Wall time: 10.6 s


' En daar is Sarah weer met een venijnig schot! Met een snelheid van 110 km/uur jaagt ze de bal richting het doel. Helaas gaat haar poging net naast. Sarah blijft knap tweede in het klassement met al 18 schoten op haar naam. Ze zal ongetwijfeld nog een paar keer haar kans wagen vandaag. Wat een power bij deze dame!'

In [42]:
%%time
call_llm_bedrock_claude_endpoint(payload,"claude-v1")

CPU times: user 3.53 ms, sys: 0 ns, total: 3.53 ms
Wall time: 4.03 s


" Sarah steps up for her 19th shot of the day. She looks focused and determined. Sarah takes a few quick steps and FIRES! The ball rockets off her boot at an incredible 110 km/h! What power and technique! The ball sails through the air, but oh no, it's just off target, missing the goal by a mere 20 meters. So close! Sarah will have to settle for 2nd place on the leaderboard for now. What a thriller here today folks,"

In [43]:
%%time
call_llm_bedrock_claude_endpoint(payload,"claude-v2")

CPU times: user 3.55 ms, sys: 0 ns, total: 3.55 ms
Wall time: 10.9 s


" Sarah lines up for her 18th shot of the day. She winds up and strikes the ball with incredible power! The shot rockets off her foot at a blistering 110 km/h! But despite the sheer velocity, the shot veers wide, missing the goal from 20 meters out. Oh, so close! Sarah was hoping to move up from her 2nd place ranking, but this errant shot means she'll have to keep fighting. With shots that fast though, you know she has what"